# Text generation with an RNN

Projext Objectives:

- Generate text using RNN.
- Creat a training examples and targets for text generation.
- Build a RNN model for serquence generation using Keras Subclassing.
- Create a text generator and evaluate the output.



This projext demonstrates how to generate text using character-based RNN. We will work with a dataset of Shakespeare's writing from Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

Bellow is a sample output when the model in this tutorial trained for 30 epochs and started with prompt "Q"

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>


While some of the sentences are grammatical, most odo not make sense. The model has not learned the meaning of words, but there are some things to consider.

  * The  model is charcter-based. When training started the model did not know how to spell an English-word, or that words were even a unit of text.

  * The strucuter of the outputresembles a play-blocks of text generally begin with a speaker name, in all the capital letters similar to the dataset.

  * As demonstrated below, the model is trained on small batches of text (100 cahracters each), and is stil able to generate a longer sequence of text with coherent strucutre.

## Setup
Import TensorFlow and other libraries

In [1]:
import os
import time
import numpy as np
import tensorflow as tf

### Download the Shakespeare dataset

In [2]:
path_to_file = tf.keras.utils.get_file(
    'shakespeare.txt',
    'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt',
)

### Read the Data

In [3]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print(f"Lenght of text: {len(text)} characters")

Lenght of text: 1115394 characters


Let's take a look at the first 250 characters in text

In [4]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



Let's see how many unique characters are in our documnet

In [5]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


This number represents the tokens that the Neural Network will consume during training, and will be generating during use

## Process the text

#### Vectorize the text
Before training, we need to convert the strings to a numerical representation.

Using `tf.keras.layers.StringLookup` layer can convert each character into an numeric ID, it just needs the text to be split into tokens first

In [6]:
example_text = ['stefan', 'nafets']

chars = tf.strings.unicode_split(example_text, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b's', b't', b'e', b'f', b'a', b'n'],
 [b'n', b'a', b'f', b'e', b't', b's']]>

Now create the `tf.keras.layers.StringLookup` layers, this will help us to tranform the character into numbers.

In [7]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab),
    mask_token=None
)

it converts from tokens to character ID's

In [8]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[58, 59, 44, 45, 40, 53],
 [53, 40, 45, 44, 59, 58]]>

Since the goal of this project is to generate text, it will be important to invert this representation and recover human-readable strings from it. To achieve this we can use `tf.keras.layers.StringLookup(......, invert=True)`.

NOTE: Here instade of pasing the original vocabulary henerated with `sorted(set(list))` use the `get_vocabulary()` method so the `tf.keras.layers.StringLookup` layer so that the `[UNK]` are set the same way.

In [9]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(),
    invert=True,
    mask_token=None
)

This layer recovers the cahracters from the vectors of ID's, and returns them as a `tf.RaggedTensor` of characters

In [10]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b's', b't', b'e', b'f', b'a', b'n'],
 [b'n', b'a', b'f', b'e', b't', b's']]>

You can use `tf.strings.reduce_join()` to join the characters back into strings


In [11]:
tf.strings.reduce_join(chars, axis=1).numpy()

array([b'stefan', b'nafets'], dtype=object)

In [12]:
# lets create a function that we can call later
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=0)

## The prediciton task

Given a character, or a sequence of characters, what is the most probable next character?

This is the task that we training the model to perform. The input to the model will be sequence of characters, and you train the model to predict the output-the following character at each time step.

Since RNN's maintain an internal state that depends on the prevours seen elements, given all the characters computed untill this moment, what is the next character?

## Create training examples and targets

Next devide the text into examples sequences. Each input sequence will containe `seq_length` characters from the text.

For each input sequence, the corresponding targets containe the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is 'Hello'. The input sequence would be 'Hell', and the target sequnce 'ello'.

First use the `tf.data.Dataset.from_tensor_slices` fucntion to convert text vector into a stream of character indices.

In [13]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [14]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [15]:
for ids in ids_dataset.take(10):
  print(chars_from_ids(ids).numpy().decode('UTF-8'))

F
i
r
s
t
 
C
i
t
i


2024-12-22 00:34:01.493958: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [16]:
seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)

The `batch` method lets you easily convert these individual characters to sequences of the desired size

In [17]:
sequences = ids_dataset.batch(seq_length + 1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


2024-12-22 00:34:06.820399: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Is easier to see what this is doing if you join the tokens beack into strings

In [18]:
#better
for seq in sequences.take(5):
    seq = tf.squeeze(seq)  # Remove unnecessary dimensions if needed
    print(text_from_ids(seq).numpy())


b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [20]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


2024-12-22 00:13:33.896746: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


For training we will need a dataset of `(input, label)` pairs. Where `input` and `label` are sequences. At each time stap the inpout is the current character and the label is the next character

The following function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestamp:

In [19]:
def split_input_target(sequence):
  input_text = sequence[:-1]
  target_text = sequence[1:]
  return input_text, target_text

In [20]:
split_input_target(list('Tensorflow'))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

We can see that in the frist row we have the input without the target and in the secound row we have the target without the label, which makes input label pairs

In [21]:
dataset = sequences.map(split_input_target)

In [22]:
for input_example, target_example in dataset.take(1):
  print('Input  :', text_from_ids(input_example).numpy())
  print('Target :', text_from_ids(target_example).numpy())

Input  : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target : b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


2024-12-22 00:34:23.911084: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


#### Create training batches
We used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches

In [23]:
# Batch size
BATCH_SIZE = 63

"""Buffer size to shuffle the dataset
(TF data is designed to work wiht possobly infinite sequences,
so it dosen't attempt to shuffle the entire sequence in memory. Instadfe
it maintaines a buffer is which it shuffles elements)"""
BUFFER_SIZE = 10000

dataset = (
    dataset.shuffle(BUFFER_SIZE) \
    .batch(BATCH_SIZE, drop_remainder=True) \
    .prefetch(tf.data.experimental.AUTOTUNE)
)
dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(63, 100), dtype=tf.int64, name=None), TensorSpec(shape=(63, 100), dtype=tf.int64, name=None))>

## Build The Model

This section defines the model as `keras.Model` sublcass.

The model have the following layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup label that will map each character-ID to a vector with `embedding_dim` dimensions.
* `tf.keras.layers.GRU`: A type of RNN with seize `units=rnn_units` (We can also use LSTM layer here)
* `tf.keras.layers.Debse`: The output layer, with `vocab_size` outputs. it outputs one logit for each character in vocabulary. There are the log-likehood of each character accoriding to the model.

In [24]:
# Length of the vocabulart in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

The class bellow does the following::
  - We derive a class from the keras.Model
  - The constructor is used to define the layers of the model
  - We define the pass forward using the layers defined in the constructor
  

In [26]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__()

        # Define embedding, GRU, and Dense layers
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(
            rnn_units,
            return_sequences=True,
            return_state=True
        )
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        # Apply embedding layer
        x = self.embedding(inputs, training=training)

        # Initialize the GRU state if it's None
        if states is None:
            batch_size = tf.shape(inputs)[0]  # Dynamically infer the batch size
            states = tf.zeros((batch_size, self.gru.units))  # Initialize states as zeros

        # Process inputs and states through the GRU layer
        x, states = self.gru(x, initial_state=states, training=training)

        # Apply Dense layer for final output
        x = self.dense(x, training=training)

        # Return the appropriate outputs based on return_state
        if return_state:
            return x, states
        else:
            return x


In [27]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size = len(ids_from_chars.get_vocabulary()),
    embedding_dim = embedding_dim,
    rnn_units = rnn_units,
)

For each character the model looks up the embedding, runs GRU one timestamp with the embedding as input, and applies the dnese layer to generate logits predicting the top-likelihhood of the next character

### Try The Model
Now run the model to see that it behaves as expected.

Frist check the shape of the output.

In [28]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(
        example_batch_predictions.shape,
        "# (batch_size, sequence_length, vocab_size)",
    )

(63, 100, 66) # (batch_size, sequence_length, vocab_size)


In the above example for sequence length of the input is `100` but the model can be run on inputs of any length


In [29]:
model.summary()

Model: "my_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (63, 100, 256)         │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ ((63, 100, 1024), (63, │     3,938,304 │
│                                 │ 1024))                 │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (63, 100, 66)          │        67,650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,022,850 (15.35 MB)

 Trainable params: 4,022,850 (15.35 MB)

 Non-trainable params: 0 (0.00 B)

To get the actual prediction from the model we need to sample from the output distribution, to get actual character indicies. The distribution is defined by the logits over hte character vocabulary

Note: It is important to sample from the distribution as taking the argmax of the distribution can easily get the model stuck in a loop

Try it for the first example in the batch

In [30]:
sampled_indices = tf.random.categorical(
    example_batch_predictions[0],
    num_samples = 1
)

sampled_indices = tf.squeeze(sampled_indices, axis=1).numpy()

This gives us, at each timestep of the next character index:

In [31]:
sampled_indices

array([55, 32, 50, 19, 28, 17, 31, 30,  5, 43, 15, 64, 18, 15,  9, 59, 47,
       44, 51, 56, 19, 30, 38,  9, 19, 64, 29, 44,  3, 58, 37, 57, 55, 12,
       24, 20, 12, 18,  1, 59,  3, 46, 52, 13, 63, 37, 17, 50, 16, 54, 20,
       15, 53, 36, 30, 39, 17, 35, 36,  5, 37, 32,  9, 64,  5,  3,  8, 12,
        3, 63, 43, 62, 65, 19, 49, 56, 45, 37, 31, 17, 15, 54, 18, 37, 42,
       30, 63, 11, 11, 35, 57, 20, 63, 15, 58, 60, 22, 65, 48, 22])

Decode these to see the text predicted by this untrained model

In [32]:
print('Input:', text_from_ids(input_example_batch[0]).numpy(), sep='\n')
print('Next Char Predictions:', text_from_ids(sampled_indices).numpy(), sep='\n')

Input:
b'y friend.\n\nAUTOLYCUS:\nAdieu, sir.\n\nFLORIZEL:\nO Perdita, what have we twain forgot!\nPray you, a word.'
Next Char Predictions:
b'pSkFODRQ&dByEB.thelqFQY.FyPe!sXrp;KG;E\nt!gm?xXDkCoGBnWQZDVW&XS.y&!-;!xdwzFjqfXRDBoEXcQx::VrGxBsuIziI'


From this output of the we can see that the model is untrained and don't understand the text and can't make accurate predictions

### Train The Model
At this point the problem can be treated as a standard classification problem. Given the previous RNN statem and the input this time step, predict the class of the next character

#### Attach an optimizer, and loss function
The standard `tf.keras.losses.sparse_categorical_crossentropy` loss funciton works in this case because it is applied across the last dimension of the predictions

Beacuse the model returns logits, you need to set the `from_logits` flag.


In [33]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [34]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print(
    'Prediciton shape: ',
    example_batch_predictions.shape,
    '# (batch_size, sequence_length, vocab_size',
    sep='\n'
)
print('New loss:        ', example_batch_mean_loss)

Prediciton shape: 
(63, 100, 66)
# (batch_size, sequence_length, vocab_size
New loss:         tf.Tensor(4.189311, shape=(), dtype=float32)


A newly initialized model should't be to sure of itself, the output logits should all have similar magnitudes. To confirm this you can chekc that the exmponential of the mean loss is approximatly equal to the vocabulary size. A much higher loss means the model is sure of it's wrong answers and is badly initialized

In [35]:
tf.exp(example_batch_mean_loss).numpy()

65.97732

Note: This value can from the examples it was evaluated on

In [36]:
model.compile(optimizer='adam', loss=loss)

#### Configure checkpoints
We will use `tf.keras.callbacks.ModelChekpoint` to ensure that checkpoints are saved during training.

In [128]:
import os
import tensorflow as tf
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Create dummy data (replace with your actual dataset)
X_train = np.random.randint(0, 100, size=(1000, 50))  # 1000 samples, each with 50 features (sequence length)
y_train = np.random.randint(0, 10, size=(1000,))  # 1000 labels for classification (10 possible classes)

# Ensure the directory for checkpoints exists
checkpoint_path = './training_checkpoints'
if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)

checkpoint_prefix = os.path.join(checkpoint_path, "ckpt_{epoch}.weights.h5")  # Add .weights.h5 extension

# Define the model (this should be replaced with your actual model)
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=100, output_dim=63),  # Example input dimension, adjust as needed
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')  # Output for 10 classes
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Create the training dataset (you can replace this with your actual dataset)
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.batch(32)  # Batch size of 32

# Create the ModelCheckpoint callback
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix, 
    save_weights_only=True,  # Save only the weights
    save_freq='epoch'  # Save checkpoint at the end of every epoch
)

# Train the model with the checkpoint callback
history = model.fit(
    train_dataset,
    epochs=10,  # Number of epochs for training
    callbacks=[checkpoint_callback]  # Pass the checkpoint callback
)
model.summary()

Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.1325 - loss: 2.3020
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.1284 - loss: 2.2915
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.1601 - loss: 2.2765
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.2184 - loss: 2.2396
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.2186 - loss: 2.1714
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.2709 - loss: 2.0813
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.2960 - loss: 1.9826
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.3289 - loss: 1.8840
Epoch 9/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.3776 - loss: 1.8012
Epoch 10/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.4047 - loss: 1.7286


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)         │ (None, 50, 63)         │         6,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_8 (GRU)                     │ (None, 128)            │        74,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 267,956 (1.02 MB)

 Trainable params: 89,318 (348.90 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 178,638 (697.81 KB)

In [39]:
checkpoint_path = './training_checkpoints'
# Name of the checkpoint file with .weights.h5 extension
checkpoint_prefix = os.path.join(checkpoint_path, "ckpt_{epoch}.weights.h5")

# Create the ModelCheckpoint callback
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix, 
    save_weights_only=True,  # Save only the weights
    save_freq='epoch'  # Save checkpoint at the end of every epoch
)


## Fit the Model
To keep training time resonable, uses 10 epochs to train the model.

Note: Set the runtime to GPU for faster training

In [40]:
# 1
for input_example_batch, target_example_batch in dataset.take(1):
    print("Input shape:", input_example_batch.shape)
    print("Target shape:", target_example_batch.shape)

Input shape: (63, 100)
Target shape: (63, 100)


In [131]:
 # depreceted
# 2
def split_input_target(chunk):
    input_text = chunk[:-1]  # All tokens except the last
    target_text = chunk[1:]  # All tokens except the first
    return input_text, tf.argmax(target_text, axis=-1)


In [132]:
 # depreceted
# Apply the function to create input-target pairs
def split_input_target(sequence):
    """
    Splits input sequence into input and target pairs.
    For example, if the input is "hello", the function returns
    input: "hell" and target: "ello".
    """
    input_text = sequence[:-1]  # All tokens except the last
    target_text = sequence[1:]  # All tokens except the first
    return input_text, target_text


In [133]:
 # depreceted
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [134]:
# Inspect dataset shapes
for input_example_batch, _ in dataset.take(1):
    print("Input example batch shape:", input_example_batch.shape)
    print("Model expects input shape:", model.input_shape)
    # If needed, reshape the input to match model's expectation
    if input_example_batch.shape[-1] != model.input_shape[-1]:
        input_example_batch = tf.reshape(input_example_batch, (-1, *model.input_shape[1:]))
        print("Reshaped input batch shape:", input_example_batch.shape)

    # Predict and verify
    example_batch_predictions = model(input_example_batch)
    print("Output shape from model:", example_batch_predictions.shape)


Input example batch shape: (63, 63, 100)
Model expects input shape: (None, 50)
Reshaped input batch shape: (7938, 50)
Output shape from model: (7938, 10)


2024-12-21 23:59:21.799685: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [88]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [89]:
 # depreceted
for input_example_batch, target_example_batch in dataset.take(1):
    print("Input shape:", input_example_batch.shape)
    print("Target shape:", target_example_batch.shape)


In [92]:
 # depreceted
# Step 1: Preprocessing the Data
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences

# Define the maximum length (50 tokens)
MAX_LENGTH = 50

# Pad and truncate the input data
for input_example_batch, target_example_batch in dataset.take(1):
    print("Original Input shape:", input_example_batch.shape)
    print("Original Target shape:", target_example_batch.shape)
    
    # Pad and truncate the input and target sequences to length 50
    input_example_batch = pad_sequences(input_example_batch, maxlen=MAX_LENGTH, padding='post', truncating='post')
    target_example_batch = pad_sequences(target_example_batch, maxlen=MAX_LENGTH, padding='post', truncating='post')
    
    print("Modified Input shape:", input_example_batch.shape)
    print("Modified Target shape:", target_example_batch.shape)

    print("Padded Input shape:", input_example_batch.shape)
    print("Padded Target shape:", target_example_batch.shape)


In [ ]:
 # depreceted
import time  # Standard Python library for timing

# Step 2: Training the Model
EPOCHS = 10

# Start measuring time
start_time = time.time()

# Train the model
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

# Calculate the time taken
end_time = time.time()
execution_time = end_time - start_time

print(f"Training completed in {execution_time:.2f} seconds.")


In [41]:
%%time
EPOCHS = 10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 162s 919ms/step - loss: 3.0685
Epoch 2/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 173s 986ms/step - loss: 1.9097
Epoch 3/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 182s 1s/step - loss: 1.6235
Epoch 4/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 189s 1s/step - loss: 1.4765
Epoch 5/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 186s 1s/step - loss: 1.3892
Epoch 6/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 315s 2s/step - loss: 1.3249
Epoch 7/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 177s 1s/step - loss: 1.2786
Epoch 8/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 242s 1s/step - loss: 1.2314
Epoch 9/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 188s 1s/step - loss: 1.1862
Epoch 10/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 190s 1s/step - loss: 1.1448
CPU times: user 2h 32min 9s, sys: 19min 1s, total: 2h 51min 11s
Wall time: 33min 25s


## Generate Text
The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as we execute it.

Each time we call the model we pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to countinue generating text.

In [42]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        # Create a mask to prevent "[UNK]" from being generated.
        skip_ids = self.ids_from_chars(["[UNK]"])[:, None]
        sparse_mask = tf.SparseTensor(
            # Put a -inf at each bad index.
            values=[-float("inf")] * len(skip_ids),
            indices=skip_ids,
            # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())],
        )
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, "UTF-8")
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits]
        predicted_logits, states = self.model(
            inputs=input_ids, states=states, return_state=True
        )
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits / self.temperature
        # Apply the prediction mask: prevent "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

        # Return the characters and model state.
        return predicted_chars, states

In [43]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the genereaated text, you'll see that model knows when to capitalize, make paragraps and imitates a Shakespare-like writing vocabulary. With the small number of training epochs, it has not yet learne to form coherent sentences.

In [44]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(
        next_char, states = states
    )
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), "\n\n" + "_" * 80)
print("\nRun time:", end - start)

ROMEO:
Back, let me see the earth, so much prepared
In act mirthre spices fortes of bows;
And, for not, and therefore, so. Now, brother
Temple themselves, the promptune of the kind
Of my person's brother's letter, pursued
For she wash the hour. Come hither, you shall not: O, Warwick, we will
one with this beauty; yet, to be found the cause
Of his land a mother confix it on,
We mird my strengthen blows out to confirm't.

GLOUCESTER:
My gracious left, for thou contagate a
cow, i' Henrou to a whole designinity-rich palm
For claim the people, even to my fortune;
Cannot stand upon thy fiving Lord Aubilit,
Here in piercing master lips. Of Henry!
How fortune save an end o' knee, wrink'd
Which thou wouldst kill'd her month-house-ruin
Matcliff'd. Wile no offection and your good,
When take he findless, make me fast, their sonows:
A duelles, most perform like the day,
Found lost her safely in the ground of ninmpel, have proposite
To usure me for the gates of my heed;
And, match Greet and my falon

The easies thing you can do to imporve the results is to trian it for longer (try `EPOCHS = 30`).

We can also experiment with diffenret start stirng, try adding another RNN layer to imporve the model's accuracy, or adjust the temperature parameter to generate more or less random predictons.

If we want the model to generate text faster the easiest thing we can do is batch the next generation in the example below the model generates 5 output in about the same time it took to generate 1 above

In [45]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO', 'ROMEO', 'ROMEO', 'ROMEO', 'ROMEO' ])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(
        next_char, states = states
    )
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + "_" * 80)
print("\nRun time", end - start)

tf.Tensor(
[b"ROMEO:\nHe's a foe intend to strengthen'd by a thousand death.\nMy sleep'st thing of all three fentless have known\nThou canst unfidectly, and hereway;\nA business of the kinghom that same vow\nLet me can apply him. Pray you, sir: I have needs must up.\n\nPEONTES:\nUnless\nI thank you off a wife:\nThese great care, disonder pride me: we will come,\nOr let's have counted. Would ye we heard you would,\nThat common vialing forticulate hate the curious servant\nA destinew and idle and a house:\nBesides his hand. And whom take him, pardon' her!\n\nLord Mayor:\nThy eyes sit would leave you!\n\nAUTOLYCUS:\nAward, fair Juckins!\n\nBENVOLIO:\nWouldst those countryman? is Some govern'd;\nBe gvoice his own himself from him!\n\nFirst Citizen:\nI must abbuly fire me to thy father.\nWho dancles hopely the kites of all the world?\n\nGLOUCESTER:\nAnd so stopp'd, add husian so too? There be\nDid pardon alive.\n\nJULIET:\nO seek there, he says love; how true loved good,\nWhich you sir, yet

## Export the generator
This signal-step nodel can easily be saved and restored, allowing you to use it anywhere a `tf.saved_model` is accepted

In [46]:
# save the one step model
tf.saved_model.save(one_step_model, 'one_step')

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [47]:
# reload the model
one_step_reloaded = tf.saved_model.load('one_step')

In [48]:
states =  None
next_char = tf.constant(['ROMEO'])
result = [next_char]

for n in range(100):
    next_char, states = one_step_reloaded.generate_one_step(
        next_char, states = states
    )
    result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
And yet he is left one hour that shall never say
He must not make it so blind a humour.
And you wi


## Advanced: Customized Training

The above training procedure is simple, but it does not give us much control, it uses teacher-forcing which prevents bad predicitons from being fed back to the model, so the model never learns to recover form mistakes.

So now that we've seen how to run the model manualy next we can implement the tarining loop. This gives a starting point if, for example, we want to implement curriculum learning to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function

We use `tf.GradientTape` to track the gradients. More about this approach :  [eager execution guide](https://www.tensorflow.org/guide/basics).

The basic procedure is

1. Execute the model and calculate the loss under a `tf.GradientTape`.
2. Calculate the updates and aply them to the model using optimizer.


In [49]:
class CustomTraining(MyModel):
    @tf.function
    def train_step(self, inputs):
        inputs, labels = inputs
        with tf.GradientTape() as tape:
            predictions = self(inputs, training=True)
            loss = self.loss(labels, predictions)
        grads = tape.gradient(loss, model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

        return {'loss': loss}

The above implementation of the `train_step` method follows `Keras train_step conventions`. This is optional, but it allows us to change the begavior of the train step and still use keras `Model.cmpile` and `Model.fit` methods.

In [50]:
model = CustomTraining(
    vocab_size = len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units = rnn_units
)

In [51]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
)

In [52]:
model.fit(dataset, epochs=1)

175/175 ━━━━━━━━━━━━━━━━━━━━ 175s 988ms/step - loss: 2.4950
